In [25]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
plt.rc('font', family='D2Coding')
from plotnine import *
import folium
from folium.plugins import MiniMap
from geopy.geocoders import Nominatim
import time
import requests

In [26]:
house_dongbuk = pd.read_csv('./data/h동북부실거래가전처리.csv')
house_dongbuk = house_dongbuk.drop(columns=['Unnamed: 0'], axis=1)
house_dongbuk

,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명,주소,위도,경도,행정동,면적(㎡)당금액
0,강북구,경남아너스빌(1356-0),114.922,89000,2003,오패산로30길 30,강북구 오패산로30길 30,37.616356,127.035671,송중동,781.0
1,강북구,경남아너스빌(1356-0),59.964,77000,2003,오패산로30길 30,강북구 오패산로30길 30,37.616356,127.035671,송중동,1305.0
2,강북구,경남아너스빌(1356-0),84.873,85000,2003,오패산로30길 30,강북구 오패산로30길 30,37.616356,127.035671,송중동,1012.0
3,강북구,꿈의숲롯데캐슬,84.980,113000,2017,오현로 56,강북구 오현로 56,37.618412,127.036476,송중동,1345.0
4,강북구,꿈의숲롯데캐슬,104.530,117000,2017,오현로 56,강북구 오현로 56,37.618412,127.036476,송중동,1125.0
...,...,...,...,...,...,...,...,...,...,...,...
3204,중랑구,한신아파트(103~109),59.760,73000,1997,동일로 752,중랑구 동일로 752,37.597009,127.082071,중화1동,1237.0
3205,중랑구,한신아파트(103~109),59.760,74000,1997,동일로 752,중랑구 동일로 752,37.597009,127.082071,중화1동,1254.0
3206,중랑구,한신아파트(103~109),84.030,91500,1997,동일로 752,중랑구 동일로 752,37.597009,127.082071,중화1동,1089.0
3207,중랑구,한영(101),84.690,49900,2003,동일로144길 74,중랑구 동일로144길 74,37.606514,127.082177,중화1동,594.0


In [27]:
house_dongbuk.columns

Index(['시군구', '단지명', '전용면적(㎡)', '거래금액(만원)', '건축년도', '도로명', '주소', '위도', '경도',
       '행정동', '면적(㎡)당금액'],
      dtype='object')

In [28]:
house_dongbuk.drop(['전용면적(㎡)', '도로명', '주소'] ,axis=1, inplace=True)
house_dongbuk

,시군구,단지명,거래금액(만원),건축년도,위도,경도,행정동,면적(㎡)당금액
0,강북구,경남아너스빌(1356-0),89000,2003,37.616356,127.035671,송중동,781.0
1,강북구,경남아너스빌(1356-0),77000,2003,37.616356,127.035671,송중동,1305.0
2,강북구,경남아너스빌(1356-0),85000,2003,37.616356,127.035671,송중동,1012.0
3,강북구,꿈의숲롯데캐슬,113000,2017,37.618412,127.036476,송중동,1345.0
4,강북구,꿈의숲롯데캐슬,117000,2017,37.618412,127.036476,송중동,1125.0
...,...,...,...,...,...,...,...,...
3204,중랑구,한신아파트(103~109),73000,1997,37.597009,127.082071,중화1동,1237.0
3205,중랑구,한신아파트(103~109),74000,1997,37.597009,127.082071,중화1동,1254.0
3206,중랑구,한신아파트(103~109),91500,1997,37.597009,127.082071,중화1동,1089.0
3207,중랑구,한영(101),49900,2003,37.606514,127.082177,중화1동,594.0


In [29]:
house_dongbuk.columns

Index(['시군구', '단지명', '거래금액(만원)', '건축년도', '위도', '경도', '행정동', '면적(㎡)당금액'], dtype='object')

In [30]:
house_dongbuk = house_dongbuk.groupby(['시군구', '단지명', '위도', '경도', '행정동', '건축년도'])['면적(㎡)당금액'].mean().round().reset_index()
house_dongbuk

,시군구,단지명,위도,경도,행정동,건축년도,면적(㎡)당금액
0,강북구,FIRSTVIEW,37.647560,127.015220,수유2동,2010,638.0
1,강북구,경남아너스빌,37.638005,127.024225,수유3동,2005,804.0
2,강북구,경남아너스빌(1356-0),37.616356,127.035671,송중동,2003,1033.0
3,강북구,궁전,37.647272,127.012891,우이동,2003,438.0
4,강북구,극동,37.641791,127.012306,인수동,1992,696.0
...,...,...,...,...,...,...,...
783,중랑구,현대,37.575176,127.089222,면목7동,1994,892.0
784,중랑구,현대아이파크,37.610695,127.073480,묵2동,2002,976.0
785,중랑구,현대휴앤미(102동),37.603025,127.084498,중화1동,2007,697.0
786,중랑구,현대휴온,37.592240,127.078707,면목2동,2013,648.0


In [31]:
house_dongbuk.to_csv('./data/w동북부실거래가단지.csv')
house_dongbuk

,시군구,단지명,위도,경도,행정동,건축년도,면적(㎡)당금액
0,강북구,FIRSTVIEW,37.647560,127.015220,수유2동,2010,638.0
1,강북구,경남아너스빌,37.638005,127.024225,수유3동,2005,804.0
2,강북구,경남아너스빌(1356-0),37.616356,127.035671,송중동,2003,1033.0
3,강북구,궁전,37.647272,127.012891,우이동,2003,438.0
4,강북구,극동,37.641791,127.012306,인수동,1992,696.0
...,...,...,...,...,...,...,...
783,중랑구,현대,37.575176,127.089222,면목7동,1994,892.0
784,중랑구,현대아이파크,37.610695,127.073480,묵2동,2002,976.0
785,중랑구,현대휴앤미(102동),37.603025,127.084498,중화1동,2007,697.0
786,중랑구,현대휴온,37.592240,127.078707,면목2동,2013,648.0
